## MotifEnumeration
Implement MotifEnumeration (reproduced below).

    Input: Integers k and d, followed by a collection of strings Dna.
    Output: All (k, d)-motifs in Dna.

---
**Sample Input:**

3 1

ATTTGGC

TGCCTTA

CGGTATC

GAAAATT

---
**Sample Output:**

ATA ATT GTT TTT

In [1]:
def HammingDistance(p, q):
    num = 0
    for i in range(len(p)):
        if p[i] != q[i]:
            num += 1
    return num

In [2]:
def ApproximatePatternCount(Text, Pattern, d):
    count = 0
    for i in range(len(Text) - len(Pattern) + 1):
        if HammingDistance(Text[i:i + len(Pattern)], Pattern) <= d:
            count += 1
    return count

In [3]:
def Neighbors(Pattern, d):
    if d == 0:
        return {Pattern}
    if len(Pattern) == 1:
        return {'A', 'C', 'G', 'T'}
    Neighborhood = set()
    suff = Pattern[1:]
    SuffixNeighbors = Neighbors(suff, d)
    for neib in SuffixNeighbors:
        if HammingDistance(neib, suff) < d:
            for x in ['A', 'C', 'G', 'T']:
                Neighborhood.add(x + neib)
        else:
            Neighborhood.add(Pattern[0] + neib)
    return list(Neighborhood)

In [4]:
def MotifEnumeration(dna, k, d):
    Patterns = set()
    string = ''.join(dna)
    for i in range(len(string) - k + 1):
        for pat in Neighbors(string[i:i + k], d):
            if all(ApproximatePatternCount(x, pat, d) for x in dna):
                Patterns.add(pat)
    return list(Patterns)

In [5]:
k = 3
d = 1
dna = ['ATTTGGC', 'TGCCTTA', 'CGGTATC', 'GAAAATT']
MotifEnumeration(dna, k, d)

['GTT', 'ATA', 'TTT', 'ATT']

In [6]:
with open('data/dataset_156_8.txt', 'r') as f:
    k, d = map(int, f.readline().strip().split(' '))
    dna = list(map(lambda x: x.strip(), f.readlines()))
print(' '.join(MotifEnumeration(dna, k, d)))

CAATT TGGAC GACAA GACCA GGACT ACAAT GGACC GGACG GGACA


## MedianString

     Input: An integer k, followed by a collection of strings Dna.
     Output: A k-mer Pattern that minimizes d(Pattern, Dna) among all k-mers Pattern. (If there are multiple such strings Pattern, then you may return any one.)
     
---
**Sample Input:**

3

AAATTGACGCAT

GACGACCACGTT

CGTCAGCGCCTG

GCTGAGCACCGG

AGTACGGGACAG

---
**Sample Output:**

ACG

In [7]:
def NumberToPattern(num, k):
    slovar = {0: "A", 1: "C", 2: "G", 3: "T"}
    pattern = ''
    while num > 3:
        pattern += slovar[num % 4]
        num = num // 4
        if num < 4:
            pattern += slovar[num]
    return 'A' * (k - len(pattern)) + pattern[::-1]

In [8]:
def Distance(Pattern, Dna):
    dist = 0
    for string in Dna:
        dist_s = len(string)
        for i in range(len(string) - len(Pattern) + 1):
            if HammingDistance(Pattern, string[i:i + len(Pattern)]) < dist_s:
                dist_s = HammingDistance(Pattern, string[i:i + len(Pattern)])
        dist += dist_s
    return dist

In [9]:
def MedianString(Dna, k):
    disnance = len(Dna[0])
    med = ''
    for i in range(4 ** k):
        patt = NumberToPattern(i, k)
        d = Distance(patt, Dna)
        if disnance > d:
            disnance = d
            med = patt
    return med

In [10]:
k = 3
dna = ['AAATTGACGCAT','GACGACCACGTT','CGTCAGCGCCTG','GCTGAGCACCGG','AGTACGGGACAG']
MedianString(dna, k)

'ACG'

In [11]:
with open('data/dataset_158_9.txt', 'r') as f:
    k = int(f.readline().strip())
    dna = list(map(lambda x: x.strip(), f.readlines()))
MedianString(dna, k)

'CATGAA'

##  Greedy Motif Search

Given a profile matrix Profile, we can evaluate the probability of every k-mer in a string Text and find a Profile-most probable k-mer in Text, i.e., a k-mer that was most likely to have been generated by Profile among all k-mers in Text. For example, ACGGGGATTACC is the Profile-most probable 12-mer in GGTACGGGGATTACCT. Indeed, every other 12-mer in this string has probability 0. In general, if there are multiple Profile-most probable k-mers in Text, then we select the first such k-mer occurring in Text.

Profile-most Probable k-mer Problem: Find a Profile-most probable k-mer in a string.

    Input: A string Text, an integer k, and a 4 × k matrix Profile.
    Output: A Profile-most probable k-mer in Text.
    
---

**Sample Input:**

    ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT
    5
    0.2 0.2 0.3 0.2 0.3
    0.4 0.3 0.1 0.5 0.1
    0.3 0.3 0.5 0.2 0.4
    0.1 0.2 0.1 0.1 0.2

---
**Sample Output:**

    CCGAG
---

In [13]:
def Profile(matrix, k_mer):
    score = 1
    for i in range(len(k_mer)):
        score *= matrix[k_mer[i]][i]
    return score

In [14]:
matrix = {
    'A': [0.2, 0.2, 0.3, 0.2, 0.3],
    'C': [0.4, 0.3, 0.1, 0.5, 0.1],
    'G': [0.3, 0.3, 0.5, 0.2, 0.4],
    'T': [0.1, 0.2, 0.1, 0.1, 0.2]
}
k_mer = 'CCGAG'
Profile(matrix, k_mer)

0.0048000000000000004

In [15]:
def ProfileMostProbableKmer(text, k, profile):
    prob = 0
    k_best = ''
    for i in range(len(text) - k + 1):
        k_mer = text[i:i + k]
        if Profile(profile, k_mer) > prob:
            prob = Profile(profile, k_mer)
            k_best = k_mer
    return k_best

In [ ]:
text = 'ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
ProfileMostProbableKmer(text, 5, matrix)

In [16]:
text = 'ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
ProfileMostProbableKmer(text, 5, matrix)

'CCGAG'